# Shaarang Buckal

Build an image classifier using the VGG16 pre-trained model, and evaluate it and compare its performance to the ResNet50 model built in the last module using the ResNet50 pre-trained model

## Part 1

Design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

In [69]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [4]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [6]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [7]:
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


58892288/58889256 [==============================] - 19s 0us/step


In [8]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 1

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)


Epoch 1/1
301/301 [==============================] - 23416s 78s/step - loss: 0.1258 - accuracy: 0.9568 - val_loss: 1.4305e-06 - val_accuracy: 0.9948


In [9]:
model.save('classifier_VGG16_model.h5')

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


## Part 2

In [15]:
model_resnet=keras.models.load_model('classifier_resnet_model.h5')

C:\Users\bucka\Anaconda3\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [63]:
test_data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle= False)


Found 500 images belonging to 2 classes.


In [22]:
resnet_score=model_resnet.evaluate_generator(test_generator)

In [26]:
model_VGG16=keras.models.load_model('classifier_VGG16_model.h5')
vgg16_score=model_VGG16.evaluate_generator(test_generator)

C:\Users\bucka\Anaconda3\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


### The VGG16 model outperformed the ResNet50 model during the training  

In [70]:
print("Resnet Model performance : ",resnet_score[1])
print("VGG16 Model performance : ", vgg16_score[1])

Resnet Model performance :  0.906000018119812
VGG16 Model performance :  0.9940000176429749


<a id="item44"></a>

## Part 3

In [43]:
vgg16_predictions=model_VGG16.predict_generator(test_generator)

In [71]:
resnet50_predictions=model_resnet.predict_generator(test_generator)

In [74]:
print("VGG16 Predictions on the first five images:",["negative" if image_prediction[0]>image_prediction[1] else "positive" for image_prediction in vgg16_predictions[0:5]])

VGG16 Predictions on the first five images: ['negative', 'negative', 'negative', 'negative', 'negative']


In [75]:
print("ResNet50 Predictions on the first five images:",["negative" if image_prediction[0]>image_prediction[1] else "positive" for image_prediction in resnet50_predictions[0:5]])

ResNet50 Predictions on the first five images: ['negative', 'negative', 'negative', 'negative', 'negative']


#### The first five Images were fetched from the negative folder that contains images with no cracks
### Both the models accurately classified higher probablity to the True Image Class